# 1. QLM syntax
## 1.1 Circuits
Let's get familiar with myQLM/QLM syntax and workflow. We will create a circuit that generates the Bell pair
$$|\psi\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}$$.
You will need to:
- create a Program() object
- allocate a 2-qubit register
- apply gates
- export to a Circuit() format
- instantiate a QPU
- create a job
- submit your job
- display the state in a text format like + 0.7 |10> - 0.7 |01>

Feel free to copy/paste from the documentation and tutorials, but spend some time to understand what each step is doing.

## 1.2 Observable

The QLM is quite handy to estimate the expectation value of given observables. Here is the energy of our pair of qubits:

$$H = J Z_0 Z_1 + h (X_0 + X_1)$$

Estimate $\langle\psi|H|\psi\rangle$:
- create an Observable() object
- create a new job (from the same circuit) of type 'OBS'
- submit it
- print the field .value 

# Quantum Approximation Optimization Algorithm (QAOA)

We will see how to implement an hybrid quantum-classical algorithm and use it to solve Combinatorial Optimization (CO) problems.

You can read the documents given with this notebook

## Max-cut problems

Maxcut is a simple CO problem which consists in finding the partition of a graph that maximize the cut.

myQLM/QLM comes with a set of classes that contain the circuit and the observable implementation. 

Let's create a graph and use the QLM classes to find the best cut.

In [ ]:
# Misc imports
import numpy as np
import matplotlib.pyplot as plt
import scipy

import networkx as nx


In [ ]:
graph = nx.generators.random_graphs.erdos_renyi_graph(7, .55)

N_nodes = len(graph.nodes)
pos=nx.spring_layout(graph)

plt.figure(figsize=(4,3))
nx.draw(graph, with_labels=True, font_weight='bold', pos=pos)


In [ ]:
def display_results(circuit):
    sampling_job = circuit.to_job()

    sol_res = qpu.submit(sampling_job)
    
    prob_sorted_states = sorted(sol_res, key = lambda s: s.probability, reverse=True)
    print("5 most probable states are:")
    for sample in prob_sorted_states[:5]:
        print(sample.state, "{:.2f}%".format(100 * sample.probability))

    # Picking the most probable cut
    best_cut = prob_sorted_states[0].state.value[0]
    print("Most probable cut:", 
        [i for i in graph.nodes() if best_cut[i] == '1'], 
        [i for i in graph.nodes() if best_cut[i] == '0'])
    
    # Plotting the cut nicely
    plt.figure(figsize=(4,3))
    nx.draw(
        graph, labels={i: str(i) for i in graph.nodes()}, pos=pos,
        node_color=['yellow' if b == '1' else '#6495ED' for b in best_cut], #limegreen
        edge_color=['red' if best_cut[a] != best_cut[b] else 'black' for a, b in graph.edges()])

## CombinatorialProblem classes

The QLM ships classes that provide all the objects needed to solve some very common combinatorial problems. Let's have a look at the Max-cut problem that consists in partition

In [ ]:
from qat.vsolve.qaoa import MaxCut as MaxCut_QAOA

Using the functions of the MaxCut class, 

_instantiate a MaxCut object and display the circuit that generates the QAOA ansatz._

_Can you recognize blocks of gates? Can you understand their roles?_

## Create an execution stack

The execution stack consists in a series of plugins piped to a qpu. Let's use ScipyMinimizePlugin to optimize the parameters of the ansatz

Instantiate a QPU and some plugin(s).

Submit the job and print the result.

What is the meaning of this result?

The graph can be drawn with the associated cut.

Build an optimal circuit using the optimal parameters (look at the meta_data property of your result) and the bind_variables function of the class Circuit().

Use the function display_results() to draw the graph

## Under the hood


In this section, we will understand what MaxCut() does internally. Feel free to read the documents given with this notebook to understand QAOA.

Let's start with the cost function. 

Can you write a function O({z}) that counts the number of of edges that connect nodes of different subset. Here we can associate a 'binary' value (-1 or 1) to each node and consider a subset the ensemble of node that have the same value. 

$$ O(\{z\}) = $$ 

Solving MaxCut means maximizing O() or minimizing -O(). 

_Can you simplify C(z)=-O(z) and create the associated Observable object with myQLM/QLM?_

In [ ]:
# Programming imports
from qat.lang.AQASM import Program, QRoutine, RX, RZ, CNOT, H
from qat.core.variables import Variable


We now implement QAOA with QLM. 

Replace the XXX in the code and comments below.  


In [ ]:
prog = XXX()
reg = prog.qalloc(XXX)
p = 2
gammas = [prog.new_var(XXX, "\\gamma_{}".format(i)) for i in range(p)]
betas = [prog.new_var(XXX, "\\beta_{}".format(i)) for i in range(p)]
for i in range(p):
    #Exploration
    for j, qb in enumerate(reg):
        prog.apply(XXX(XXX),XXX)
        
    #Exploitation
    for edge in graph.edges:
        prog.apply(XXX, XXX, XXX)
        prog.apply(XXX(XXX), XXX)
        prog.apply(XXX, XXX, XXX)
        
abstract_circuit=prog.XXX()


Simulate and analyze the results as we did before.

# Graph coloring

Max-Cut problems can be seen as finding the best 2-color graph coloring. Let's try to find a QAOA implementation for the same problem but with k colors

For this part, you can read (Hadfield et al., 2019) https://arxiv.org/pdf/1709.03489.pdf for help. Pages 10-12 and 38 are particularly helpful. Do not hesitate to browse through the introduction (section 2 and 3).  

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import numpy.random as npr
import networkx as nx
import copy

from qat.lang.AQASM import Program,QRoutine,RY,RX,RZ, H,CNOT,Z,PH, X
from qat.core import Observable, Term
from qat.qpus import get_default_qpu

from qat.lang.AQASM.gates import ParamGate, AbstractGate
from qat.lang.AQASM.misc import build_gate
from qat.core.variables import Variable
from qat.core.circuit_builder.matrix_util import default_gate_set

In [ ]:
qpu = get_default_qpu()

## The one-hot encoding

One-hot encoding is a way to encode information on a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

In the following, we will use this encoding to color the graph given at the beginning of this challenge. What is the main drawback of this encoding? 

Let's consider nb_c=3 colors. How many qubits will we need?

In [ ]:
nb_c = 3

Write in latex how a cost function that would do the job?

$$ C(\{Z_i\}) = $$

_Write it then with myQLM_

## Step 1: Exploitation
Create a function that implements the phase-separation operator.
    
You can start by creating a subcircuit that does this for a single edge (get inspiration from what you did with MaxCut). 

Then build a second a function that implements this step for all edges of the graph. 

Feel free to display the qroutine for each step

## Step 2: Exploration
_Create a function that implements the mixing operator_

We give you the one_qubit_mixing gate as a matrix. Working with a simulator offers the freedom to bypass some gate implementation and use directly arbitrary operators.

_What is this gate?_ 

hint: you can use a qpu to understand its effect. Use bind_variables({variables_names: values}) to assign values to program variables created with .new_var()

In [ ]:
def one_qubit_mixing(theta):
    return np.array([[1, 0, 0, 0],
                     [0, np.cos(theta), 1j*np.sin(theta), 0],
                     [0, 1j*np.sin(theta), np.cos(theta), 0],
                     [0, 0, 0, 1]])

one_qubit_mixing = AbstractGate("1QUBT_MIXING", [float], arity=2, matrix_generator=one_qubit_mixing)
    
gs=default_gate_set()
gs.add_signature(one_qubit_mixing)


With the one one_qubit_mixing gate, _write first a function that implements the mixing operator $U_v$ which 'mixes' one node._

The $U_M$ mixer operator mixes all the nodes to create a large superposition of (valid with respect to the one-hot encoding) states

_Write the qroutine associated to $U_M$._

Optionnal: with the help of (Hadfield et al., 2019), you could find a quantum routine that do the job of one_qubit_mixing()

## Step 3:
_Write the full program that will create the QAOA ansatz. Don't forget a coherent initialization._

## Simulation

Once your program is done, export it in the circuit format and use it in a simulation to extract the optimal $\gamma_i$ and $\beta_i$

In [ ]:
method="Nelder-Mead"
tol=1e-2
options={"maxiter":2}

## Analysis

The simulation must have returned a Results() object with the optimal $\gamma_i$ and $\beta_i$. _Extract them (use the properties .meta_data)._

_Write a function similar to display_results that draw a colored graph_ 

In [ ]:
def display_results_multicolor(params):
    c_res = circ.bind_variables(params,gate_set=gs)

    #... 

In [ ]:
display_results_multicolor(XXX)

Congratulation, you've made it! You have implemented QAOA for graph coloring. 